This is a placeholder file aiming to remind us that:

Althought there seems to be no big issue in the csv file,
it is important to preprocess the images even before the feature extraction!

I've found an interesting article:
https://medium.com/towards-data-science/medical-image-pre-processing-with-python-d07694852606

 - 2023/11/29 I haven't found time to read it yet.